In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!rm -rf "drive/My Drive/Colab Notebooks/pp/split_complete"

In [11]:
!ls "drive/My Drive/Colab Notebooks/pp/split_complete/"

ls: cannot access 'drive/My Drive/Colab Notebooks/pp/split_complete/': No such file or directory


In [0]:
!unzip -uq "drive/My Drive/Colab Notebooks/pp/split.zip" -d "drive/My Drive/Colab Notebooks/pp/"

In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision

In [4]:
!pip install Pillow==4.1.1
!pip install PIL
!pip install image

    100% |████████████████████████████████| 5.7MB 895kB/s 
  Found existing installation: Pillow 5.2.0
    Uninstalling Pillow-5.2.0:
      Successfully uninstalled Pillow-5.2.0
  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL
    100% |████████████████████████████████| 7.3MB 3.6MB/s 


In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [0]:
def eval_model(vgg, criterion):
    since = time.time()
    avg_loss = 0
    avg_acc = 0
    loss_test = 0
    acc_test = 0
    
    test_batches = len(dataloaders[TEST])
    print("Evaluating model")
    print('-' * 10)
    
    nfp = 0
    counter = 0
    cl = 0
    lfn = 0
    ch = 0
    hfn = 0
    
    for i, data in enumerate(dataloaders[TEST]):
        if i % 100 == 0:
            print("\rTest batch {}/{}".format(i, test_batches), end='', flush=True)
           
          
        print(data)
        break

        vgg.train(False)
        vgg.eval()
        inputs, labels = data

        if use_gpu:
            inputs, labels = Variable(inputs.cuda(), volatile=True), Variable(labels.cuda(), volatile=True)
        else:
            inputs, labels = Variable(inputs, volatile=True), Variable(labels, volatile=True)

        outputs = vgg(inputs)

        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)

        loss_test += loss.data[0]
        acc_test += torch.sum(preds == labels.data)
        for p,r in zip(preds.cpu().numpy(),labels.data.cpu().numpy()):
            if p == 2 and r != 2:
              nfp += 1
            if r == 1:
              cl += 1
              if p != 1:
                lfn += 1
            if r == 0:
              ch += 1
              if p != 0:
                hfn += 1
            counter += 1

        del inputs, labels, outputs, preds
        torch.cuda.empty_cache()
        
    avg_loss = loss_test / dataset_sizes[TEST]
    avg_acc = acc_test / dataset_sizes[TEST]
    
    elapsed_time = time.time() - since
    print()
    print("Evaluation completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("Avg loss (test): {:.4f}".format(avg_loss))
    print("Avg acc (test): {:.4f}".format(avg_acc))
    print("Falsly classifed as no light: ", nfp)
    print("Low light: ", cl)
    print("Low light but not classified: ", lfn)
    print("High light: ", ch)
    print("High light but not classified: ", hfn)
    print("Complete counter: ", counter)
    print('-' * 10)

In [0]:
def train_model(vgg, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()
    best_model_wts = copy.deepcopy(vgg.state_dict())
    best_acc = 0.0
    
    avg_loss = 0
    avg_acc = 0
    avg_loss_val = 0
    avg_acc_val = 0
    
    train_batches = len(dataloaders[TRAIN])
    val_batches = len(dataloaders[VAL])
    
    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch, num_epochs))
        print('-' * 10)
        
        loss_train = 0
        loss_val = 0
        acc_train = 0
        acc_val = 0
        
        vgg.train(True)
        
            
        for i, data in enumerate(dataloaders[TRAIN]):
            if i % 50 == 0:
                print("\rTraining batch {}/{}".format(i, train_batches / 2), end='', flush=True)
                
            # Use half training dataset
            if i >= train_batches / 2:
                break
                
            inputs, labels = data
            
            
            if use_gpu:
                inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
            else:
                inputs, labels = Variable(inputs), Variable(labels)
            
            optimizer.zero_grad()
            
            outputs = vgg(inputs)
            
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            loss_train += loss.data[0]
            acc_train += torch.sum(preds == labels.data)
            
            del inputs, labels, outputs, preds
            torch.cuda.empty_cache()
        
        print()
        # * 2 as we only used half of the dataset
        avg_loss = loss_train * 2 / dataset_sizes[TRAIN]
        avg_acc = acc_train * 2 / dataset_sizes[TRAIN]
        
        vgg.train(False)
        vgg.eval()
            
        for i, data in enumerate(dataloaders[VAL]):
            if i % 100 == 0:
                print("\rValidation batch {}/{}".format(i, val_batches), end='', flush=True)
                
            inputs, labels = data
            
            if use_gpu:
                inputs, labels = Variable(inputs.cuda(), volatile=True), Variable(labels.cuda(), volatile=True)
            else:
                inputs, labels = Variable(inputs, volatile=True), Variable(labels, volatile=True)
            
            optimizer.zero_grad()
            
            outputs = vgg(inputs)
            
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            loss_val += loss.data[0]
            acc_val += torch.sum(preds == labels.data)
            
            del inputs, labels, outputs, preds
            torch.cuda.empty_cache()
        
        avg_loss_val = loss_val / dataset_sizes[VAL]
        avg_acc_val = acc_val / dataset_sizes[VAL]
        
        print()
        print("Epoch {} result: ".format(epoch))
        print("Avg loss (train): {:.4f}".format(avg_loss))
        print("Avg acc (train): {:.4f}".format(avg_acc))
        print("Avg loss (val): {:.4f}".format(avg_loss_val))
        print("Avg acc (val): {:.4f}".format(avg_acc_val))
        print('-' * 10)
        print()
        
        if avg_acc_val > best_acc:
            best_acc = avg_acc_val
            best_model_wts = copy.deepcopy(vgg.state_dict())
        
    elapsed_time = time.time() - since
    print()
    print("Training completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("Best acc: {:.4f}".format(best_acc))
    
    vgg.load_state_dict(best_model_wts)
    return vgg

In [0]:
use_gpu = True

In [9]:
data_dir = "drive/My Drive/Colab Notebooks/pp/split_complete"
print(os.path.abspath(data_dir))
TRAIN = 'train'
VAL = 'val'
TEST = 'test'
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# VGG-16 Takes 224x224 images as input, so we resize all of them
data_transforms = {
    TRAIN: transforms.Compose([
        # Data augmentation is a good practice for the train set
        # Here, we resize the image to 224x224 and
        # randomly flip it horizontally. 
        transforms.Resize(224),
        transforms.RandomVerticalFlip(),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    VAL: transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        normalize,
    ]),
    TEST: transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        normalize,
    ])
}

image_datasets = {
    x: datasets.ImageFolder(
        os.path.join(data_dir, x), 
        transform=data_transforms[x]
    )
    for x in [TRAIN, VAL, TEST]
}

dataloaders = {
    x: torch.utils.data.DataLoader(
        image_datasets[x], batch_size=8,
        shuffle=True, num_workers=4
    )
    for x in [TRAIN, VAL, TEST]
}


dataset_sizes = {x: len(image_datasets[x]) for x in [TRAIN, VAL, TEST]}

for x in [TRAIN, VAL, TEST]:
    print("Loaded {} images under {}".format(dataset_sizes[x], x))
    
print("Classes: ")
class_names = image_datasets[TRAIN].classes
print(image_datasets[TRAIN].classes)

/content/drive/My Drive/Colab Notebooks/pp/split_complete
Loaded 22595 images under train
Loaded 2908 images under val
Loaded 6779 images under test
Classes: 
['HIGH_LIGHT', 'LOW_LIGHT', 'NO_LIGHT']


In [10]:
vgg16 = models.vgg16(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.torch/models/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:09<00:00, 60584069.02it/s]


In [11]:
# Freeze training for all layers
for param in vgg16.features.parameters():
    param.require_grad = False
# print(vgg16)

# # Newly created modules have require_grad=True by default
# num_features_1 = vgg16.classifier[0].out_features
# features = list(vgg16.classifier.children())[:-4] 
# features.extend([nn.Linear(num_features_1, 500)]) # Add our layer with 500 outputs
# features.extend([nn.ReLU(inplace=True)])
# features.extend([nn.Dropout(p=0.5)]) 
# features.extend([nn.Linear(500, len(class_names))]) # Add our layer with 3 outputs

num_features = vgg16.classifier[6].in_features
features = list(vgg16.classifier.children())[:-1] 
features.extend([nn.Linear(num_features, len(class_names))]) # Add our layer with 3 outputs
vgg16.classifier = nn.Sequential(*features) # Replace the model classifier
print(vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (5): Conv2d (64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (10): Conv2d (128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (17): Conv2d (256, 512, kernel_size=(3, 3), 

In [0]:
if use_gpu:
    vgg16.cuda()
    
vgg16.load_state_dict(torch.load("drive/My Drive/Colab Notebooks/pp/VGG16.pth"))
    
# weights = [0.4,0.4,0.2]
# class_weights = torch.FloatTensor(weights).cuda()
# criterion = nn.CrossEntropyLoss(weight=class_weights)
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(vgg16.parameters(), lr=0.00001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [15]:
%matplotlib inline
print("Test before training")
eval_model(vgg16, criterion)

Test before training
Evaluating model
----------
Test batch 0/848[
( 0 , 0 ,.,.) = 
  0.6734  0.8104  0.8447  ...   0.8104  0.9132  1.2214
  0.4508  0.6734  0.9988  ...   1.2043  1.1529  0.8276
  0.6221  0.5878  0.9303  ...   1.3070  1.2728  0.2967
           ...             ⋱             ...          
  1.3584  1.3413  1.3584  ...   1.7523  1.8037  1.1700
  1.5468  1.6153  1.7180  ...   1.7694  2.0263  1.7352
  1.6324  1.6324  1.6495  ...   1.2557  1.5297  1.5297

( 0 , 1 ,.,.) = 
  0.8880  1.0805  1.1155  ...   0.9580  1.0630  1.3081
  0.6429  0.8704  1.2381  ...   1.4307  1.3606  0.9580
  0.8004  0.7829  1.1331  ...   1.5532  1.4482  0.4328
           ...             ⋱             ...          
  1.4307  1.3782  1.4132  ...   1.9384  1.9909  1.3256
  1.6057  1.7108  1.8683  ...   1.9559  2.2360  1.9034
  1.7283  1.7283  1.7983  ...   1.4482  1.6583  1.6933

( 0 , 2 ,.,.) = 
  0.9668  1.1585  1.1934  ...   1.1237  1.1934  1.3851
  0.8099  0.9668  1.3154  ...   1.4548  1.4025  1.0365


In [23]:
vgg16 = train_model(vgg16, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=15)

Epoch 0/15
----------
Training batch 1400/1412.5
Validation batch 300/364
Epoch 0 result: 
Avg loss (train): 0.0914
Avg acc (train): 0.6762
Avg loss (val): 0.0919
Avg acc (val): 0.6685
----------

Epoch 1/15
----------
Training batch 1400/1412.5
Validation batch 300/364
Epoch 1 result: 
Avg loss (train): 0.0919
Avg acc (train): 0.6762
Avg loss (val): 0.0917
Avg acc (val): 0.6688
----------

Epoch 2/15
----------
Training batch 1400/1412.5
Validation batch 300/364
Epoch 2 result: 
Avg loss (train): 0.0925
Avg acc (train): 0.6741
Avg loss (val): 0.0914
Avg acc (val): 0.6733
----------

Epoch 3/15
----------
Training batch 1400/1412.5
Validation batch 300/364
Epoch 3 result: 
Avg loss (train): 0.0913
Avg acc (train): 0.6757
Avg loss (val): 0.0910
Avg acc (val): 0.6709
----------

Epoch 4/15
----------
Training batch 1400/1412.5
Validation batch 300/364
Epoch 4 result: 
Avg loss (train): 0.0914
Avg acc (train): 0.6757
Avg loss (val): 0.0906
Avg acc (val): 0.6702
----------

Epoch 5/15
----

In [18]:
print("Test after training")
eval_model(vgg16, criterion)

Test after training
Evaluating model
----------
Test batch 800/848
Evaluation completed in 2m 47s
Avg loss (test): 0.0965
Avg acc (test): 0.6529
Falsly classifed as no light:  1226
Low light:  1689
Low light but not classified:  1211
High light:  2338
High light but not classified:  792
Complete counter:  6779
----------


In [0]:
torch.save(vgg16.state_dict(), 'drive/My Drive/Colab Notebooks/pp/VGG16_500.pth')